## Setup


In [3]:
import os
import json
import requests

API_URL = 'https://api.app.genei.io/graphql'
API_KEY = 'sk_6c34c355499a44c382c4aa0ace62b215'
HEADERS = {
    'Content-Type': 'application/json',
    'x-api-key': API_KEY
}

data_dir_path = 'data/'
dist_dir_path = 'dist/'

## Retrieve Genei note data


### Search Genei notes


In [4]:
note_name = "1.0 - Quantitative Data Analysis - 'Introduction' - Week 1 Lecture Notes"

# Search for notes in Genei by name
query = """
    query {{
        searchByName(input: {{
            name: "{}"
        }}) {{
            results {{
                item {{
                    __typename
                    ... on Note {{
                        id
                        name
                    }}
                }}
                score
            }}
        }}
    }}
""".format(note_name)
try:
    response = requests.post(API_URL, json={'query': query}, headers=HEADERS)
    # Raise an exception if the response status code is not 200
    response.raise_for_status()
    search_results = response.json()['data']['searchByName']['results']
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    search_results = []

top_search_result_id = None
if len(search_results) > 0:
    top_search_result_id = search_results[0]['item']['id']

top_search_result_id

'note_cl8vi4vr3008701kw3s6ue0y7'

### Return note data from top search result


In [5]:
if top_search_result_id:
    query = """
    query {{
        note(id: "{}") {{
            data
        }}
    }}
    """.format(top_search_result_id)
    response = requests.post(API_URL, json={'query': query}, headers=HEADERS)
    note = response.json()
    note_data = note['data']['note']['data']['root']

    if not os.path.exists(data_dir_path):
        os.makedirs(data_dir_path)

    file_name = os.path.join(data_dir_path, note_name + '.json')
    with open(file_name, 'w') as f:
        json.dump(note_data, f)

## Convert Genei note data to markdown


In [37]:
def process_text_nodes(children):
    """
    Process and combine adjacent text nodes, applying formatting where necessary.
    """
    combined_text = []
    current_text = ""

    for child in children:
        if child['type'] == 'text':
            # If the text should be bold
            if child.get('format') == 1:
                # If there's existing non-bold text, append it first
                if current_text:
                    combined_text.append(current_text)
                    current_text = ""
                combined_text.append(f"**{child['text']}**")
            else:
                # Combine adjacent non-bold text nodes
                current_text += child['text']
        else:
            # If there's existing non-bold text, append it first
            if current_text:
                combined_text.append(current_text)
                current_text = ""
            combined_text.append(json_to_md(child))

    # Append any remaining text
    if current_text:
        combined_text.append(current_text)

    return ''.join(combined_text)


def json_to_md(node, parent_list_type=None):
    """
    Convert a Genei note JSON node to Markdown.
    """
    node_type = node['type']

    if node_type in ['text', 'code-highlight']:
        node_text = node.get('text')
        if node_text and node_text != '[link]':
            return f"**{node_text}**" if node.get('format') == '1' else node_text

    if node_type == 'heading':
        level = int(node['tag'][1])
        return f"{'#' * level} " + ' '.join([json_to_md(child) for child in node['children']]) + "\n"

    if node_type == 'list':
        list_type = node.get('listType', 'bullet')
        return '\n'.join([json_to_md(child, list_type) for child in node['children']])

    if node_type == 'listitem':
        if parent_list_type == 'bullet':
            prefix = "- "
        else:
            prefix = f"{node['value']}. "
        indent = node.get('indent', 0)
        prefix = '\t' * indent + prefix

        combined_text = process_text_nodes(node['children'])
        if node['children'] and node['children'][0].get('type') == 'list':
            return combined_text
        else:
            return prefix + combined_text

    if node_type == 'code':
        return f"```{node.get('language', '')}\n" + ''.join([json_to_md(child) for child in node['children']]) + "\n```"

    if node_type == 'equation':
        sign = '$' if node.get('inline', False) else '$$'
        return sign + node.get('equation', '') + sign

    if node_type == 'quote':
        return '> ' + ' '.join([json_to_md(child) for child in node['children']])

    if node_type == 'resource-link':
        return ' '.join([json_to_md(child) for child in node['children']])

    if node_type in ['paragraph', 'root']:
        return '\n'.join([json_to_md(child) for child in node['children']])

    if node_type == 'linebreak':
        return '\n'

    return ''


markdown_output = json_to_md(note_data)

if not os.path.exists(dist_dir_path):
    os.makedirs(dist_dir_path)

file_name = os.path.join(dist_dir_path, note_name + '.md')
with open(file_name, 'w') as f:
    f.write(markdown_output)